## This script uses Scanpy to preprocess scRNA-seq data, then applies LIANA to perform communication inference for each sample, and constructs a 4-dimensional communication score tensor.

In [1]:
import pandas as pd
import scanpy as sc
import liana as li
import warnings
warnings.filterwarnings('ignore')

%matplotlib inline

# 1. Preprocessing Data

## 1.1. Download Data

**RNA-seq data**

Expression matrix was obtained from: https://cells.ucsc.edu/autism/exprMatrix.tsv.gz
<br>Values are 10x UMI counts from cellranger, log2-transformed

Metadata was obtained from: https://cells.ucsc.edu/autism/meta.tsv

Here is the bash script to download data:

```
raw_data_dir=/home/qdai8/projects/Projects/STDCC/Data/RDA/ASD/raw
mkdir -p ${raw_data_dir}
cd ${raw_data_dir}
wget https://cells.ucsc.edu/autism/exprMatrix.tsv.gz 
wget https://cells.ucsc.edu/autism/meta.tsv
wget --no-check-certificate https://cells.ucsc.edu/autism/rawMatrix.zip
```

In [2]:
data_folder = '/home/qdai8/projects/Projects/STDCC/Data/RDA/ASD/raw/rawMatrix'
# Load the raw data
adata = sc.read_mtx(f'{data_folder}/matrix.mtx').T  # Transpose because the data is in cells x genes format

# Load gene names
genes = pd.read_csv(f'{data_folder}/genes.tsv', header=None, sep='\t')
genes.columns = ['gene_id', 'gene_symbol']
genes['combined'] = genes['gene_id'] + '|' + genes['gene_symbol']
adata.var_names = genes['combined'].values

# Load cell barcodes
barcodes = pd.read_csv(f'{data_folder}/barcodes.tsv', header=None, sep='\t')
adata.obs_names = barcodes[0].values

# Load metadata
metadata = pd.read_csv(f'{data_folder}/meta.txt', sep='\t')

# Set metadata into adata.obs, ensuring that the barcodes match
adata.obs = metadata.set_index('cell')

In [3]:
print(adata)

AnnData object with n_obs × n_vars = 104559 × 65217
    obs: 'cluster', 'sample', 'individual', 'region', 'age', 'sex', 'diagnosis', 'Capbatch', 'Seqbatch', 'post-mortem interval (hours)', 'RNA Integrity Number', 'genes', 'UMIs', 'RNA mitochondr. percent', 'RNA ribosomal percent'


## 1.2 Select PFC brain region

In [4]:
brain_area = 'PFC'
filtered_adata = adata[adata.obs.region == brain_area]

In [5]:
print(filtered_adata)

View of AnnData object with n_obs × n_vars = 62166 × 65217
    obs: 'cluster', 'sample', 'individual', 'region', 'age', 'sex', 'diagnosis', 'Capbatch', 'Seqbatch', 'post-mortem interval (hours)', 'RNA Integrity Number', 'genes', 'UMIs', 'RNA mitochondr. percent', 'RNA ribosomal percent'


## 1.3 Basic Quality Control

In [6]:
# filter cells and genes
sc.pp.filter_cells(filtered_adata, min_genes=200)
sc.pp.filter_genes(filtered_adata, min_cells=3)
# log1p normalize the data
sc.pp.normalize_total(filtered_adata)
sc.pp.log1p(filtered_adata)

In [7]:
print(filtered_adata)

AnnData object with n_obs × n_vars = 62166 × 36745
    obs: 'cluster', 'sample', 'individual', 'region', 'age', 'sex', 'diagnosis', 'Capbatch', 'Seqbatch', 'post-mortem interval (hours)', 'RNA Integrity Number', 'genes', 'UMIs', 'RNA mitochondr. percent', 'RNA ribosomal percent', 'n_genes'
    var: 'n_cells'
    uns: 'log1p'


In [8]:
filtered_adata.obs.head()

,cluster,sample,individual,region,age,sex,diagnosis,Capbatch,Seqbatch,post-mortem interval (hours),RNA Integrity Number,genes,UMIs,RNA mitochondr. percent,RNA ribosomal percent,n_genes
cell,,,,,,,,,,,,,,,,
AAACCTGAGATGGCGT-1_4341_BA46,AST-PP,4341_BA46,4341,PFC,13,M,Control,CB6,SB2,16,7.2,792,1121,0.089206,1.338091,792
AAACCTGAGCTAACTC-1_4341_BA46,Oligodendrocytes,4341_BA46,4341,PFC,13,M,Control,CB6,SB2,16,7.2,658,928,0.431034,0.862069,658
AAACCTGAGTGTTAGA-1_4341_BA46,L5/6,4341_BA46,4341,PFC,13,M,Control,CB6,SB2,16,7.2,4890,13640,0.857771,0.469208,4890
AAACCTGCACCCATTC-1_4341_BA46,L2/3,4341_BA46,4341,PFC,13,M,Control,CB6,SB2,16,7.2,3967,8526,0.469153,0.516069,3967
AAACCTGTCTCATTCA-1_4341_BA46,Microglia,4341_BA46,4341,PFC,13,M,Control,CB6,SB2,16,7.2,621,846,0.236407,0.945627,621


In [9]:
filtered_adata.var.head()

,n_cells
ENSG00000227232|WASH7P,1902
ENSG00000243485|RP11-34P13.3,34
ENSG00000238009|RP11-34P13.7,410
ENSG00000233750|CICP27,781
ENSG00000268903|RP11-34P13.15,198


In [10]:
filtered_adata.var.index = [idx.split('|')[1] for idx in filtered_adata.var.index]

In [11]:
filtered_adata.var.head()

,n_cells
WASH7P,1902
RP11-34P13.3,34
RP11-34P13.7,410
CICP27,781
RP11-34P13.15,198


# 2. Use LIANA to Perform CCC Inference by Sample

In [12]:
sample_key = 'sample'
groupby = 'cluster'

In [13]:
li.mt.rank_aggregate.by_sample(
    filtered_adata,
    groupby=groupby,
    resource_name='consensus', # NOTE: uses human gene symbols!
    sample_key=sample_key, # sample key by which we which to loop
    use_raw=False,
    verbose=True, # use 'full' to show all verbose information
    n_perms=None, # exclude permutations for speed
    return_all_lrs=True, # return all LR values
    )

Converting `sample` to categorical!
Now running: 6033_BA9: 100%|████████████████████| 23/23 [18:59<00:00, 49.54s/it]


- Check LIANA results

In [14]:
filtered_adata.uns["liana_res"].sort_values("magnitude_rank").head(10)

,sample,source,target,ligand_complex,receptor_complex,lr_means,expr_prod,scaled_weight,lr_logfc,spec_weight,lrscore,lr_probs,magnitude_rank
4067097,5387_BA9,Oligodendrocytes,L4,IL1RAPL1,PTPRD,2.749131,7.532037,1.341422,1.574888,0.013149,0.985969,0.315791,9.016331e-17
0,4341_BA46,Oligodendrocytes,L5/6,IL1RAPL1,PTPRD,2.909284,8.453215,1.234929,1.877218,0.018843,0.986918,0.301844,9.712150e-17
11237476,5577_BA9,Oligodendrocytes,L5/6,IL1RAPL1,PTPRD,2.958418,8.747614,1.299729,2.050151,0.015614,0.987234,0.342469,9.832725e-17
9161589,5538_PFC_Nova,Oligodendrocytes,L5/6,IL1RAPL1,PTPRD,2.546762,6.485092,1.146841,1.639261,0.017277,0.984700,0.225071,1.026221e-16
3024096,5294_BA9,Oligodendrocytes,L5/6,IL1RAPL1,PTPRD,2.939969,8.642257,1.423251,1.901085,0.016388,0.986878,0.327244,1.028995e-16
10210659,5565_BA9,Oligodendrocytes,L4,IL1RAPL1,PTPRD,2.910561,8.470068,1.402330,1.746054,0.015712,0.985576,0.376755,1.101020e-16
16427627,5936_PFC_Nova,Oligodendrocytes,L5/6,IL1RAPL1,PTPRD,2.652002,6.979914,1.378636,1.697211,0.014790,0.984870,0.287358,1.114811e-16
14392200,5879_PFC_Nova,Oligodendrocytes,L5/6,IL1RAPL1,PTPRD,2.683192,7.198975,1.341997,1.754207,0.017267,0.984811,0.292105,1.116390e-16
15406590,5893_PFC,Oligodendrocytes,L5/6,IL1RAPL1,PTPRD,2.748686,7.554482,0.975239,1.620620,0.017941,0.986031,0.266091,1.123799e-16
17451265,5939_BA9,Oligodendrocytes,L5/6,IL1RAPL1,PTPRD,2.954045,8.721391,1.280630,1.495094,0.014238,0.985974,0.402054,1.190898e-16


- Check number of CCC events by sample

In [15]:
filtered_adata.uns["liana_res"]['sample'].value_counts()

sample
5387_BA9         1096755
5841_BA9         1080282
4341_BA46        1072768
5958_BA9         1062075
5577_BA9         1058029
5531_BA9         1049070
5538_PFC_Nova    1049070
5408_PFC_Nova    1043290
5294_BA9         1043001
5565_BA9         1026817
5936_PFC_Nova    1023638
5893_PFC         1021037
5864_BA9         1016413
5879_PFC_Nova    1014390
5939_BA9         1007454
6033_BA9         1002252
5278_PFC          990981
5945_PFC          974219
5144_PFC          960347
5403_PFC          956012
5419_PFC          949365
5976_BA9          698175
5978_BA9          676350
Name: count, dtype: int64

- Save LIANA results

In [16]:
out_dir = '/home/qdai8/projects/Projects/STDCC/Data/RDA/ASD/raw/'

In [17]:
filtered_adata.write_h5ad(out_dir + "liana_res.h5ad")

... storing 'cluster' as categorical
... storing 'region' as categorical
... storing 'sex' as categorical
... storing 'diagnosis' as categorical
... storing 'Capbatch' as categorical
... storing 'Seqbatch' as categorical


# 3. Build Tensor

- Use expr_prod as the communication score function to construct tensor

In [18]:
tensor = li.multi.to_tensor_c2c(filtered_adata,
                                sample_key=sample_key,
                                score_key='expr_prod', # can be any score from liana
                                how='outer_cells',
                                outer_fraction=0.8)

100%|██████████████████████████████████████████| 23/23 [56:48<00:00, 148.21s/it]


In [19]:
tensor.tensor.shape

(23, 2610, 17, 17)

- Save constructed tensor

In [20]:
import os
out_path = '/home/qdai8/projects/Projects/STDCC/Data/RDA/ASD/LIANA_expr_prod'
if not os.path.exists(out_path):
    os.makedirs(out_path)

In [21]:
from scipy.io import savemat
mymat={'c2ctensor':tensor.tensor}
savemat(out_path + "/c2ctensor.mat", mymat)

- Save dimension names (ligand-receptor pairs, sender cell types, receiver cell types)

In [22]:
pd.DataFrame(tensor.order_names[1]).to_csv(out_path + '/lr.txt', 
                      mode='w',
                      index=None,
                      header=None,
                      sep='\t')

pd.DataFrame(tensor.order_names[2]).to_csv(out_path + '/sender.txt', 
                      mode='w',
                      index=None,
                      header=None,
                      sep='\t')

pd.DataFrame(tensor.order_names[3]).to_csv(out_path + '/receiver.txt', 
                      mode='w',
                      index=None,
                      header=None,
                      sep='\t')

- Save individual-level information (e.g. disease status, age, gender) for subjects

In [23]:
subjects = tensor.order_names[0]
out_cols = ['sample', 'individual', 'age', 'sex', 'diagnosis', 'Seqbatch']
subject_info = filtered_adata.obs[out_cols].drop_duplicates().reset_index(drop=True)


# Step 1: Create a mapping from list values to order indices
order_mapping = {value: index for index, value in enumerate(subjects)}
# Step 2: Map the 'cov_ind' column to order indices
subject_info['order'] = subject_info['sample'].map(order_mapping)
# Step 3: Sort the DataFrame based on the 'order' column
subject_info = subject_info.sort_values('order').drop(columns='order').reset_index(drop=True)



In [24]:
subject_info

,sample,individual,age,sex,diagnosis,Seqbatch
0,4341_BA46,4341,13,M,Control,SB2
1,5144_PFC,5144,7,M,ASD,SB1
2,5278_PFC,5278,15,F,ASD,SB1
3,5294_BA9,5294,19,M,ASD,SB2
4,5387_BA9,5387,12,M,Control,SB2
5,5403_PFC,5403,16,M,ASD,SB1
6,5408_PFC_Nova,5408,6,M,Control,SB1
7,5419_PFC,5419,19,F,ASD,SB1
8,5531_BA9,5531,15,M,ASD,SB2
9,5538_PFC_Nova,5538,19,F,Control,SB1


In [25]:
subject_info.to_csv(out_path + '/subjects_info.txt', 
                    mode='w',
                    index=None,
                    header=True,
                    sep='\t')